In [1]:
import os

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Project\\e2e\\ml_practice\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Project\\e2e\\ml_practice'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_dir: Path
    unzip_dir: Path
    

In [11]:
from mlProject.constants import *
from mlProject.utils.comman import read_yaml, create_directories



In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath  = CONFIG_FILE_PATH, params_filepath =  PARAMS_FILE_PATH, schema_filepath =  SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params =  read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config =  DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_dir=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    
    


In [17]:
import os
import urllib.request as request
import zipfile
from mlProject.utils import logger
from mlProject.utils.comman import get_size

In [19]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def dowload_file(self):
        if not os.path.exists(self.config.local_dir):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_dir
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_dir))}")

    def extract_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_dir,'r') as zip_ref: zip_ref.extractall(unzip_path)


In [22]:
try:
    config = ConfigurationManager()
    dataingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(dataingestion_config)
    data_ingestion.dowload_file()
    data_ingestion.extract_file()
except Exception as e:
    raise e

[2023-07-29 12:55:12,092: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-07-29 12:55:12,097: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-07-29 12:55:12,101: INFO: comman: yaml file: schema.yaml loaded successfully]
[2023-07-29 12:55:12,104: INFO: comman: created directory at: artifacts]
[2023-07-29 12:55:12,107: INFO: comman: created directory at: artifacts/data_ingestion]
[2023-07-29 12:55:13,117: INFO: 690454767: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "51b18a12af535f5d423d2095fe7eaec31d09f53a2a8dd7c27dfd4ba408337832"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 33D8:26C04F:1B6725:26245D:64C4BED4
Accept-Ranges: bytes
Date: Sat, 29 